<a href="https://colab.research.google.com/github/yukinaga/ai_music/blob/main/section_3/01_polyphony_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Polyphony RNNによる作曲
「Polyphony RNN」により、バッハ風の曲を生成しましょう。

## ライブラリのインストール
Magentaと共に、音楽生成用のライブラリpyFluidSynth、MIDIデータを処理するためのpretty_midiなどをインストールします。

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

## 起点となるメロディの設定
RNNの起点となるNoteSequenceを設定します。  

In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

seed = music_pb2.NoteSequence()  # NoteSequence

# notesにnoteを追加
seed.notes.add(pitch=80, start_time=0.0, end_time=0.4, velocity=80)
seed.notes.add(pitch=80, start_time=0.4, end_time=0.8, velocity=80)
seed.notes.add(pitch=87, start_time=0.8, end_time=1.2, velocity=80)
seed.notes.add(pitch=87, start_time=1.2, end_time=1.6, velocity=80)
seed.notes.add(pitch=89, start_time=1.6, end_time=2.0, velocity=80)
seed.notes.add(pitch=89, start_time=2.0, end_time=2.4, velocity=80)
seed.notes.add(pitch=87, start_time=2.4, end_time=3.2, velocity=80)

seed.total_time = 3.2  # 所要時間
seed.tempos.add(qpm=75);  # 曲のテンポを指定

note_seq.plot_sequence(seed)  # NoteSequenceの可視化
note_seq.play_sequence(seed, synth=note_seq.fluidsynth)  # NoteSequenceの再生

## Polyphony RNNの初期化

Polyphony RNNは、LSTMをベースにバッハ風の曲を生成することができます。  
合唱曲、すなわち和音をつないで進行する曲が生成されます。

https://github.com/magenta/magenta/tree/main/magenta/models/polyphony_rnn

学習済みのモデルは Bundleファイル（.magファイル）に保存されています。  
以下のコードは、学習済みモデル「polyphony_rnn.mag」を読み込み、曲の生成器を設定しています

In [ ]:
from magenta.models.polyphony_rnn import polyphony_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# モデルの初期化
note_seq.notebook_utils.download_bundle("polyphony_rnn.mag", "/models/")  # Bundle（.magファイル）をダウンロード
bundle = sequence_generator_bundle.read_bundle_file("/models/polyphony_rnn.mag")  # Bundleの読み込み
generator_map = polyphony_sequence_generator.get_generator_map()
polyphony_rnn = generator_map["polyphony"](checkpoint=None, bundle=bundle)  # 生成器の設定
polyphony_rnn.initialize()  # 初期化

## 曲の生成

各設定を行った上で、生成器により曲を生成します。  
`temperature`を変更することで、曲の「ランダム度合い」を調整することができます。

In [ ]:
from note_seq.protobuf import generator_pb2

total_time = 180 # 曲の長さ（秒）
temperature = 1.0 # 曲の「ランダム度合い」を決める定数

base_end_time = max(note.end_time for note in seed.notes)  #ベース曲の終了時刻

# 生成器に関する設定
generator_options = generator_pb2.GeneratorOptions()  # 生成器のオプション
generator_options.args["temperature"].float_value = temperature  # ランダム度合い
generator_options.generate_sections.add(
    start_time=base_end_time,  # 作曲開始時刻
    end_time=total_time)  # 作曲終了時刻

# 曲の生成
gen_seq = polyphony_rnn.generate(seed, generator_options)

note_seq.plot_sequence(gen_seq)  # NoteSequenceの可視化
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)  # NoteSequenceの再生

## MIDIファイルの保存とダウンロード
`NoteSequence`をMIDIデータに変換し、保存してダウンロードします。


In [ ]:
from google.colab import files

note_seq.sequence_proto_to_midi_file(gen_seq, "polyphony_rnn.mid")  #MIDI　データに変換し保存
files.download("polyphony_rnn.mid")  # ダウンロード